# 📊 Parcial – Análisis de Redes Sociales (Primer Corte)

## 🎯 Objetivo
Evaluar tu dominio de **filtros** y **agrupaciones** en Python (sin pivots ni gráficos) y tu capacidad de **análisis de negocio** en métricas de marketing: **CAC, LTV, churn rate, funnel, unit economics (LTV/CAC)**.

---
## 🧩 Contexto de Negocio (Caso Real)
**StartUp SaaS en crecimiento.** Vende planes **Basic, Pro, Enterprise** en **LatAm, North America, Europe, APAC**. Los canales de adquisición son: **meta_ads, google_ads, organic_search, outbound_sales**. Los usuarios se registran por **web** o **mobile_app**.

La dirección quiere decidir **dónde invertir el presupuesto del próximo trimestre**. Te piden:

1) Identificar **canales más rentables** (relación **LTV/CAC** y churn).  
2) Detectar **regiones** con mayor **potencial de crecimiento** y/o **riesgo**.  
3) Evaluar el desempeño por **tier** (Basic, Pro, Enterprise).  
4) Recomendar **asignación de presupuesto** por **canal + tier**.

---
## 📂 Instrucciones
- Trabaja únicamente con **filtros** y **agrupaciones**. **No uses** tablas dinámicas (pivot), merges ni gráficos.
- Archivo a usar: `clientes_marketing.csv`.
- Escribe **código + interpretación breve** para cada inciso.
- Si el enunciado dice **“Agrupa”**, usa `groupby`. Si dice **“Filtra”**, usa filtrado con máscaras booleanas. Si dice **“Ambas”**, realiza primero el **filtro** y luego la **agrupación**.

---


## Descargar Tabla (Correr una vez)

In [1]:
!wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv

--2025-09-18 01:05:09--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv [following]
--2025-09-18 01:05:09--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661398 (646K) [text/plain]
Saving to: ‘cac_ltv_model.csv’

cac_ltv_model.csv   100%[===================>] 645.90K  --.-KB/s

## Importar Datos

In [2]:
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# 1) Cargar dataset
data = pd.read_csv('cac_ltv_model.csv')


### ✅ Variables relevantes (recordatorio)
- `acquisition_channel`, `signup_source`, `region`, `customer_tier`
- `plan_price`, `discount_rate`, `arpu`, `gross_margin`, `churn_rate`
- `contract_length_months`, `marketing_spend`

### 🧮 Fórmulas de negocio (a usar en el examen)
- **CAC por canal**:  
    $$ CAC = \frac{\text{Total Marketing Spend}}{\text{Clientes adquiridos}} $$
  
- **LTV por observación** (aprox.):  
   $$ LTV = ARPU \times contract\_length\_months \times gross\_margin \times (1 - churn\_rate) $$

- **Unit economics**:  
  $$ LTV/CAC   $$(ideal \> 3 como regla general de SaaS)

---



### Parte Exploratoria: Cree una muestra de los primeros 5 elementos de esta tabla. ¿Cuantas observaciones tienes?

,year,month,date,customer_id,acquisition_channel,signup_source,region,customer_tier,plan_price,discount_rate,arpu,gross_margin,churn_rate,contract_length_months,marketing_spend
0,2023,1,Jan-23,1001,outbound_sales,web,North America,Basic,78.84,0.10,63.63,0.76,0.02,12,212.48
1,2023,1,Jan-23,1002,meta_ads,web,LatAm,Pro,120.23,0.00,124.47,0.84,0.05,1,352.60
2,2023,1,Jan-23,1003,organic_search,mobile_app,North America,Enterprise,335.82,0.20,278.20,0.81,0.02,12,55.40
3,2023,1,Jan-23,1004,organic_search,web,Europe,Pro,193.89,0.10,190.65,0.80,0.08,1,49.10
4,2023,1,Jan-23,1005,organic_search,web,APAC,Enterprise,471.80,0.10,445.32,0.80,0.08,1,45.27


## 🔎 Parte A – Filtros

hint: Crea una tabla con el filtro y luego agrupa la variable que te piden.

1. **Filtra** los clientes del canal **`meta_ads`**. Calcula el **ARPU promedio** de este subconjunto.  



ARPU promedio para clientes de meta_ads: 168.79


2. **Filtra** clientes de **LatAm** con **`churn_rate > 0.05`**. ¿Cuántos son? ¿Cuál es su **ARPU promedio**?  

In [6]:
latam_high_churn = data[(data['region'] == 'LatAm') & (data['churn_rate'] > 0.05)]
num_clientes_latam_high_churn = latam_high_churn.shape[0]
arpu_promedio_latam_high_churn = latam_high_churn['arpu'].mean()

print(f"Número de clientes en LatAm con churn_rate > 0.05: {num_clientes_latam_high_churn}")
print(f"ARPU promedio para estos clientes: {arpu_promedio_latam_high_churn:,.2f}")

Número de clientes en LatAm con churn_rate > 0.05: 410
ARPU promedio para estos clientes: 174.78


3. **Filtra** clientes **Enterprise** con `contract_length_months > 6`. ¿Cuál es su **gross_margin promedio**?

In [8]:
enterprise_long_contract = data[(data['customer_tier'] == 'Enterprise') & (data['contract_length_months'] > 6)]
gross_margin_promedio_enterprise_long_contract = enterprise_long_contract['gross_margin'].mean()

print(f"Gross margin promedio para clientes Enterprise con contract_length_months > 6: {gross_margin_promedio_enterprise_long_contract:,.2f}")

Gross margin promedio para clientes Enterprise con contract_length_months > 6: 0.82


## 📊 Parte B – Agrupaciones (SOLO agrupar)
4. **Agrupa** por `acquisition_channel` y calcula **ARPU promedio**. Ordena de mayor a menor.  


In [7]:
arpu_por_canal = data.groupby('acquisition_channel')['arpu'].mean().sort_values(ascending=False)
print("ARPU promedio por canal de adquisición (ordenado de mayor a menor):")
display(arpu_por_canal)

ARPU promedio por canal de adquisición (ordenado de mayor a menor):


,arpu
acquisition_channel,
outbound_sales,173.61
google_ads,172.37
organic_search,171.14
meta_ads,168.79


5. **Agrupa** por `region` y calcula **churn_rate promedio**. Identifica la región con mayor churn.  



In [9]:
churn_por_region = data.groupby('region')['churn_rate'].mean().sort_values(ascending=False)
region_mayor_churn = churn_por_region.index[0]

print("Churn rate promedio por región (ordenado de mayor a menor):")
display(churn_por_region)
print(f"\nLa región con mayor churn es: {region_mayor_churn}")

Churn rate promedio por región (ordenado de mayor a menor):


,churn_rate
region,
Middle East,0.05
LatAm,0.05
North America,0.05
APAC,0.05
Europe,0.05
Africa,0.05



La región con mayor churn es: Middle East


6. **Agrupa** por `customer_tier` y calcula **marketing_spend total**. ¿Cuál tier consume más presupuesto?

Gasto total de marketing por nivel de cliente (ordenado de mayor a menor):


,marketing_spend
customer_tier,
Pro,"589,265.04"
Basic,"584,419.11"
Enterprise,"584,348.87"



El tier que consume más presupuesto es: Pro


## 🔀 Parte C – Filtro **y** Agrupación (Ambas)
7. **Filtra** solo registros de **`signup_source = 'web'`** y luego **agrupa** por `acquisition_channel` para obtener el **churn_rate promedio**.  


In [11]:
web_signups = data[data['signup_source'] == 'web']
churn_rate_web_by_channel = web_signups.groupby('acquisition_channel')['churn_rate'].mean()
print("Churn rate promedio por canal de adquisición para registros con signup_source = 'web':")
display(churn_rate_web_by_channel)

Churn rate promedio por canal de adquisición para registros con signup_source = 'web':


,churn_rate
acquisition_channel,
google_ads,0.05
meta_ads,0.05
organic_search,0.05
outbound_sales,0.05


8. **Filtra** solo **`mobile_app`** y luego **agrupa** por `region` para calcular **ARPU promedio**.  


In [13]:
mobile_app_signups = data[data['signup_source'] == 'mobile_app']
arpu_mobile_by_region = mobile_app_signups.groupby('region')['arpu'].mean()
print("ARPU promedio por región para registros con signup_source = 'mobile_app':")
display(arpu_mobile_by_region)

ARPU promedio por región para registros con signup_source = 'mobile_app':


,arpu
region,
APAC,169.09
Africa,172.60
Europe,165.89
LatAm,173.20
Middle East,171.71
North America,174.00


9. **Filtra** a clientes **`Pro`** y **agrupa** por `acquisition_channel` para calcular **marketing_spend total**.


In [14]:
pro_customers = data[data['customer_tier'] == 'Pro']
marketing_spend_pro_by_channel = pro_customers.groupby('acquisition_channel')['marketing_spend'].sum()
print("Gasto total de marketing para clientes Pro por canal de adquisición:")
display(marketing_spend_pro_by_channel)

Gasto total de marketing para clientes Pro por canal de adquisición:


,marketing_spend
acquisition_channel,
google_ads,"226,721.68"
meta_ads,"211,517.60"
organic_search,"29,848.72"
outbound_sales,"121,177.04"


## 📈 Parte D – Métricas de negocio (CAC, LTV, LTV/CAC)
10. **CAC por canal (Agrupa)**: calcula el CAC de cada `acquisition_channel` como:  
   `CAC = marketing_spend_total_del_canal / #clientes_del_canal`  


In [15]:
cac_por_canal = data.groupby('acquisition_channel').agg(
    marketing_spend_total=('marketing_spend', 'sum'),
    num_clientes=('customer_id', 'count')
)
cac_por_canal['CAC'] = cac_por_canal['marketing_spend_total'] / cac_por_canal['num_clientes']

print("CAC por canal de adquisición:")
display(cac_por_canal[['CAC']])

CAC por canal de adquisición:


,CAC
acquisition_channel,
google_ads,401.02
meta_ads,349.27
organic_search,49.95
outbound_sales,200.22


11. **CAC por tier (Agrupa)**: calcula el CAC de cada `customer_tier` como:  
   `CAC = marketing_spend_total_del_tier / #clientes_del_canal`  


In [16]:
cac_por_tier = data.groupby('customer_tier').agg(
    marketing_spend_total=('marketing_spend', 'sum'),
    num_clientes=('customer_id', 'count')
)
cac_por_tier['CAC'] = cac_por_tier['marketing_spend_total'] / cac_por_tier['num_clientes']

print("CAC por nivel de cliente (tier):")
display(cac_por_tier[['CAC']])

CAC por nivel de cliente (tier):


,CAC
customer_tier,
Basic,249.75
Enterprise,249.19
Pro,248.43


12. **LTV por tier (Ambas)**: crea una columna `ltv_individual` con la fórmula dada.


In [17]:
data['ltv_individual'] = data['arpu'] * data['contract_length_months'] * data['gross_margin'] * (1 - data['churn_rate'])
display(data[['customer_id', 'ltv_individual']].head())

,customer_id,ltv_individual
0,1001,567.54
1,1002,99.20
2,1003,"2,651.91"
3,1004,139.68
4,1005,328.72


12. A. Luego **agrupa** `ltv_individual` por
`acquisition_channel` para obtener el **LTV promedio**.  

In [18]:
ltv_por_canal = data.groupby('acquisition_channel')['ltv_individual'].mean()
print("LTV promedio por canal de adquisición:")
display(ltv_por_canal)

LTV promedio por canal de adquisición:


,ltv_individual
acquisition_channel,
google_ads,710.25
meta_ads,729.05
organic_search,765.13
outbound_sales,751.57


12. B. Luego **agrupa** `ltv_individual`por `customer_tier` para obtener el **LTV promedio**.





In [19]:
ltv_por_tier = data.groupby('customer_tier')['ltv_individual'].mean()
print("LTV promedio por nivel de cliente (tier):")
display(ltv_por_tier)

LTV promedio por nivel de cliente (tier):


,ltv_individual
customer_tier,
Basic,291.77
Enterprise,"1,349.02"
Pro,577.63


13. **Unit economics (Ambas)**: combina tus resultados para comparar **LTV promedio por tier** contra **CAC por canal** y comenta **qué combinaciones canal + tier** lucen más saludables (busca **LTV/CAC > 3**).


In [20]:
# Calculate LTV/CAC for each customer (using the CAC of their acquisition channel)
# We need to map the CAC per channel back to the individual customer rows
cac_map = cac_por_canal['CAC'].to_dict()
data['cac_individual'] = data['acquisition_channel'].map(cac_map)
data['ltv_cac_ratio'] = data['ltv_individual'] / data['cac_individual']

# Group by acquisition_channel and customer_tier to get the average LTV/CAC ratio
ltv_cac_by_channel_tier = data.groupby(['acquisition_channel', 'customer_tier'])['ltv_cac_ratio'].mean().unstack()

print("LTV/CAC promedio por combinación de canal de adquisición y nivel de cliente:")
display(ltv_cac_by_channel_tier)
print("\nCombinaciones canal + tier con LTV/CAC > 3:")
display(ltv_cac_by_channel_tier[ltv_cac_by_channel_tier > 3].dropna(how='all'))

LTV/CAC promedio por combinación de canal de adquisición y nivel de cliente:


customer_tier,Basic,Enterprise,Pro
acquisition_channel,,,
google_ads,0.67,3.17,1.40
meta_ads,0.90,3.88,1.60
organic_search,5.78,28.24,11.94
outbound_sales,1.46,6.80,2.97



Combinaciones canal + tier con LTV/CAC > 3:


customer_tier,Basic,Enterprise,Pro
acquisition_channel,,,
google_ads,NaN,3.17,NaN
meta_ads,NaN,3.88,NaN
organic_search,5.78,28.24,11.94
outbound_sales,NaN,6.80,NaN


## 🧠 Parte E – Análisis crítico (respuesta abierta)
13. **CMO por un día**: con tus resultados, ¿en qué **canal(es)** invertirías más el próximo trimestre? ¿Por qué? (Cita **CAC** y **churn**).  
14. ¿Qué **región** ves con mayor riesgo? ¿Qué hipótesis explicarían su **churn**?  
15. Identifica **dos combinaciones canal + tier** con mejor **LTV/CAC** y propón **dos acciones** concretas de optimización (p. ej., creatividades, audiencias, pricing, retención).

---
✍️ **Nota**: Mantente disciplinado con el enunciado (**Filtra**, **Agrupa**, **Ambas**). La evaluación pondera **correctitud técnica** y **calidad de interpretación de negocio**.
